In [304]:


import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

torch.manual_seed(42)


In [305]:


data = [ ("me gusta comer en la cafeteria".split(), "SPANISH"),
         ("Give it to me".split(), "ENGLISH"),
         ("No creo que sea una buena idea".split(), "SPANISH"),
         ("No it is not a good idea to get lost at sea".split(), "ENGLISH") ]

test_data = [ ("Yo creo que si".split(), "SPANISH"),
              ("it is lost on me".split(), "ENGLISH")]

word_to_ix = {}
for sent, _ in data + test_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print (word_to_ix)
VOCAB_SIZE = len(word_to_ix)
NUM_LABELS = 2

{'me': 0, 'gusta': 1, 'comer': 2, 'en': 3, 'la': 4, 'cafeteria': 5, 'Give': 6, 'it': 7, 'to': 8, 'No': 9, 'creo': 10, 'que': 11, 'sea': 12, 'una': 13, 'buena': 14, 'idea': 15, 'is': 16, 'not': 17, 'a': 18, 'good': 19, 'get': 20, 'lost': 21, 'at': 22, 'Yo': 23, 'si': 24, 'on': 25}


In [306]:
class BoW(nn.Module):
    def __init__(self, NUM_LABELS, VOCAB_SIZE):
        super(BoW, self).__init__()
        self.Layer1 = nn.Linear(VOCAB_SIZE, NUM_LABELS)

    def forward(self, bow_vec):
        output = F.softmax(self.Layer1(bow_vec))
        return output

In [307]:
def make_bow_vector(sentence, word_to_ix):
    vec = torch.zeros(len(word_to_ix))
    for word in sentence:
        vec[word_to_ix[word]] += 1
    return vec.view(1, -1)

def make_target(label, label_to_ix):
    return torch.LongTensor([label_to_ix[label]])

In [308]:
Model = BoW(NUM_LABELS, VOCAB_SIZE)

In [309]:
sample = data[0]
bow_vector = make_bow_vector(sample[0], word_to_ix)
out = model(autograd.Variable(bow_vector))
out

C:\Users\admin\AppData\Local\Temp\ipykernel_10376\3816136875.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = F.softmax(self.Layer1(bow_vec))


tensor([[0.6120, 0.3880]], grad_fn=<SoftmaxBackward0>)

In [310]:
label_to_ix = { "SPANISH": 0, "ENGLISH": 1 }

In [311]:
for instance, label in test_data:
    print(instance)
    print(f"model outputs: {Model(autograd.Variable(make_bow_vector(instance, word_to_ix)))}")
    print(label)
    print("-----------------")

['Yo', 'creo', 'que', 'si']
model outputs: tensor([[0.4989, 0.5011]], grad_fn=<SoftmaxBackward0>)
SPANISH
-----------------
['it', 'is', 'lost', 'on', 'me']
model outputs: tensor([[0.4931, 0.5069]], grad_fn=<SoftmaxBackward0>)
ENGLISH
-----------------


C:\Users\admin\AppData\Local\Temp\ipykernel_10376\3816136875.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = F.softmax(self.Layer1(bow_vec))


In [312]:
loss_function = nn.NLLLoss()
optimizer = optim.SGD(Model.parameters(), lr=0.1)

for epoch in range(1000):
    for instance, label in data:
        Model.zero_grad()
        bow_vec = autograd.Variable(make_bow_vector(instance, word_to_ix))
        y = autograd.Variable(make_target(label, label_to_ix))

        probs = Model(bow_vec)

        loss = loss_function(probs, y)
        loss.backward()
        optimizer.step()

C:\Users\admin\AppData\Local\Temp\ipykernel_10376\3816136875.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = F.softmax(self.Layer1(bow_vec))


In [313]:
for instance, label in test_data:
    bow_vec = autograd.Variable(make_bow_vector(instance, word_to_ix))
    log_probs = Model(bow_vec)
    print (log_probs)

tensor([[0.9433, 0.0567]], grad_fn=<SoftmaxBackward0>)
tensor([[0.0213, 0.9787]], grad_fn=<SoftmaxBackward0>)


C:\Users\admin\AppData\Local\Temp\ipykernel_10376\3816136875.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = F.softmax(self.Layer1(bow_vec))
